In [1]:
import getpass
import os
import re
import jsonify
from pathlib import Path
from typing import List
import base64
import json
from PIL import Image
import ast
import pytesseract
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key ········


In [2]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.messages.human import HumanMessage
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

In [3]:
# Simple image to string
extracted_text = pytesseract.image_to_string(Image.open('receipt2.jpg'))

In [4]:
gemini = GoogleGenerativeAI(model="gemini-1.5-pro")

In [5]:
def calculate_days_to_expiry(expiry_dates):
    today = datetime.today().date()
    return {item: (datetime.strptime(date, "%Y-%m-%d").date() - today).days for item, date in expiry_dates.items()}

In [12]:
prompt_template_items = PromptTemplate(
    input_variables=["extracted_text"],
    template="""
    this is the text extracted from a grocery bill using pytesseract:{extracted_text}. 
    Extract the list of grocery items from this. 
    Write the names of the grocery items used for food only in a readable format with full name. 
    Write these items as a list, like: ['Item 1 Name', 'Item 2 Name', 'Item 3 Name', ...] with no preamble.
    """
)

In [13]:
chain_items = prompt_template_items | gemini | StrOutputParser()

In [14]:
result = chain_items.invoke(input={"extracted_text":extracted_text})

In [15]:
print(result)

['Pasta Parmesan Pesto',
'Hellmanns Mayonnaise',
'Monster Ultra Green',
'Monster Ultra Blue',
'Mushrooms White',
'Baby Spinach',
'Carrots',
'Apple Honeycrisp',
'Banana',
'Zucchini Green',
'Snap Peas',
'Roasted Garlic Hummus',
'Cheese Halloumi',
'Liver Sausage',
'Cheddar Cheese 2 Year']


In [19]:
# Convert the result string to a Python list
grocery_list = ast.literal_eval(result)

# Convert the list to JSON
grocery_json = json.dumps({"grocery_items": grocery_list})

# Example: print or return this JSON to send to frontend
print(grocery_json)

{"grocery_items": ["Pasta Parmesan Pesto", "Hellmanns Mayonnaise", "Monster Ultra Green", "Monster Ultra Blue", "Mushrooms White", "Baby Spinach", "Carrots", "Apple Honeycrisp", "Banana", "Zucchini Green", "Snap Peas", "Roasted Garlic Hummus", "Cheese Halloumi", "Liver Sausage", "Cheddar Cheese 2 Year"]}


In [87]:
prompt_template_meal = PromptTemplate(
    input_variables=["age", "weight", "height", "gender", "diet_type", "allergies", "health_conditions", "health_goal", "cuisine", "item_list", "days_to_expiry"],
    template="""
    1. You are an AI nutritionist. Generate a 7-day meal plan (4 meals per day) based on the following details:
    - Age: {age}
    - Weight: {weight} kg
    - Height: {height} cm
    - Gender: {gender}
    - Dietary Preferences: {diet_type}
    - Allergies: {allergies}
    - Health Conditions: {health_conditions}
    - Health Goal: {health_goal}
    - Preferred Cuisine: {cuisine}
    - Available Ingredients: {item_list}
    - Time Until Expiry (Days): {days_to_expiry}
    
    2. The plan should be optimized to minimize food waste by prioritizing items that expire soon.
    3. Strictly do not repeat the exact same meals on consecutive days.
    4. Double check your responses before finalizing the meal plan.
    5. Output the plan in JSON format with days (monday, tuesday, wednesday, etc.) as keys and each day containing 4 meals (breakfast, lunch, snacks, dinner).
    6. Make sure that the response does not contain any preamble. Return JSON text only, no markdown formatting, no triple backticks, no explanations.
    """ )

In [88]:
chain_meal = prompt_template_meal | gemini | StrOutputParser()

In [89]:
result = chain_meal.invoke(input=
    {"age":25, "weight":70, "height":175, "gender":"male",
        "diet_type":"vegetarian", "allergies":"Peanuts", "health_conditions":"Diabetes",
        "health_goal":"muscle gain", "cuisine":"Indian",
        "item_list":["Pasta Parmesan Pesto", "Hellmanns Mayonnaise", "Monster Ultra Green", "Monster Ultra Blue", "Mushrooms White", "Baby Spinach", "Carrots", "Apple Honeycrisp", "Banana", "Zucchini Green", "Snap Peas", "Roasted Garlic Hummus", "Cheese Halloumi", "Liver Sausage", "Cheddar Cheese 2 Year"],
        "days_to_expiry":calculate_days_to_expiry({
  "Pasta Parmesan Pesto": "2025-04-22",
  "Hellmanns Mayonnaise": "2025-05-15",
  "Monster Ultra Green": "2025-06-01",
  "Monster Ultra Blue": "2025-06-03",
  "Mushrooms White": "2025-04-10",
  "Baby Spinach": "2025-04-12",
  "Carrots": "2025-04-20",
  "Apple Honeycrisp": "2025-04-18",
  "Banana": "2025-04-09",
  "Zucchini Green": "2025-04-14",
  "Snap Peas": "2025-04-17",
  "Roasted Garlic Hummus": "2025-04-25",
  "Cheese Halloumi": "2025-05-05",
  "Liver Sausage": "2025-04-27",
  "Cheddar Cheese 2 Year": "2025-06-10"
})}
)

In [90]:
# Clean markdown formatting
cleaned = re.sub(r"^```json|```$", "", result.strip(), flags=re.IGNORECASE).strip("`\n ")

meal_plan = json.loads(cleaned)
print(json.dumps(meal_plan, indent=2))

{
  "monday": {
    "breakfast": "Spinach and Mushroom Omelette with Halloumi",
    "lunch": "Pasta Pesto with Zucchini and Snap Peas",
    "snacks": "Apple slices with Roasted Garlic Hummus and Banana",
    "dinner": "Halloumi and Carrot Curry with Brown Rice (not from provided ingredients, needs to be acquired)"
  },
  "tuesday": {
    "breakfast": "Banana Smoothie with Spinach and Cheddar Cheese",
    "lunch": "Halloumi and Snap Pea Salad with Hummus dressing",
    "snacks": "Apple slices with Cheddar Cheese",
    "dinner": "Mushroom and Zucchini Curry with Brown Rice (not from provided ingredients, needs to be acquired)"
  },
  "wednesday": {
    "breakfast": "Scrambled Halloumi with Spinach and Mushrooms",
    "lunch": "Pasta Pesto with Carrots and Baby Spinach",
    "snacks": "Banana with a dollop of Hummus",
    "dinner": "Zucchini and Carrot Curry with Brown Rice (not from provided ingredients, needs to be acquired)"
  },
  "thursday": {
    "breakfast": "Spinach and Cheddar Ch

In [31]:
prompt_template_meal2 = PromptTemplate(
    input_variables=["age", "weight", "height", "gender", "diet_type", "allergies", "health_conditions", "health_goal", "cuisine", "item_list", "days_to_expiry"],
    template="""
    1. You are an AI nutritionist. Generate a 7-day meal plan (4 meals per day) based on the following details:
    - Age: {age}
    - Weight: {weight} kg
    - Height: {height} cm
    - Gender: {gender}
    - Dietary Preferences: {diet_type}
    - Allergies: {allergies}
    - Health Conditions: {health_conditions}
    - Health Goal: {health_goal}
    - Preferred Cuisine: {cuisine}
    - Available Ingredients: {item_list}
    - Time Until Expiry (Days): {days_to_expiry}

    2. The plan should be optimized to minimize food waste by prioritizing items that expire soon.
    3. Strictly do not repeat the exact same meals on consecutive days.
    4. Double-check your responses before finalizing the meal plan.
    5. For each meal (breakfast, lunch, snacks, dinner), provide both the recipe name and the *detailed step-by-step cooking instructions*.
    6. Output the plan in JSON format with days (monday, tuesday, wednesday, etc.) as keys and each day containing 4 meals (breakfast, lunch, snacks, dinner). For each meal, include:
       - "recipe_name": (name of the dish)
       - "instructions": (step-by-step preparation instructions in detail)
       - "how_it_helps": (how the meal helps my dietary preferences, health conditions and goals)
       - "calories": (total calories along with how many calories each item contributes per serve)
    7. Make sure that the response does not contain any preamble. Return JSON text only, no markdown formatting, no triple backticks, no explanations.
    """
)


In [32]:
chain_meal2 = prompt_template_meal2 | gemini | StrOutputParser()

In [33]:
result2 = chain_meal2.invoke(input=
    {"age":25, "weight":70, "height":175, "gender":"male",
        "diet_type":"vegetarian", "allergies":"Peanuts", "health_conditions":"Diabetes",
        "health_goal":"muscle gain", "cuisine":"Indian",
        "item_list":["Pasta Parmesan Pesto", "Hellmanns Mayonnaise", "Monster Ultra Green", "Monster Ultra Blue", "Mushrooms White", "Baby Spinach", "Carrots", "Apple Honeycrisp", "Banana", "Zucchini Green", "Snap Peas", "Roasted Garlic Hummus", "Cheese Halloumi", "Liver Sausage", "Cheddar Cheese 2 Year"],
        "days_to_expiry":calculate_days_to_expiry({
  "Pasta Parmesan Pesto": "2025-04-22",
  "Hellmanns Mayonnaise": "2025-05-15",
  "Monster Ultra Green": "2025-06-01",
  "Monster Ultra Blue": "2025-06-03",
  "Mushrooms White": "2025-04-10",
  "Baby Spinach": "2025-04-12",
  "Carrots": "2025-04-20",
  "Apple Honeycrisp": "2025-04-18",
  "Banana": "2025-04-09",
  "Zucchini Green": "2025-04-14",
  "Snap Peas": "2025-04-17",
  "Roasted Garlic Hummus": "2025-04-25",
  "Cheese Halloumi": "2025-05-05",
  "Liver Sausage": "2025-04-27",
  "Cheddar Cheese 2 Year": "2025-06-10"
})}
)

In [34]:
# Clean markdown formatting
cleaned = re.sub(r"^```json|```$", "", result2.strip(), flags=re.IGNORECASE).strip("`\n ")

meal_plan = json.loads(cleaned)
print(json.dumps(meal_plan, indent=2))

{
  "monday": {
    "breakfast": {
      "recipe_name": "Banana Spinach Smoothie",
      "instructions": "1. Blend 1 banana with a handful of baby spinach. 2. Add a splash of water for desired consistency.",
      "how_it_helps": "Provides a quick and easy diabetic-friendly breakfast with good fiber and nutrients from spinach. Supports muscle gain with the potassium in banana.",
      "calories": "Approximately 150 calories (Banana: 100, Spinach: 50)"
    },
    "lunch": {
      "recipe_name": "Halloumi and Mushroom Stir-fry",
      "instructions": "1. Slice halloumi and mushrooms. 2. Saute\u0301 mushrooms in a pan with a little olive oil until softened. 3. Add halloumi and cook until golden brown. 4. Serve with a side of baby spinach.",
      "how_it_helps": "Provides protein for muscle gain and is low-carb for diabetes management.  Uses expiring mushrooms and spinach.",
      "calories": "Approximately 400 calories (Halloumi: 250, Mushrooms: 50, Spinach: 100)"
    },
    "snacks": {


In [41]:
prompt_template_waste = PromptTemplate(
    input_variables=["item_data"],
    template="""
You are a smart grocery assistant. A user has marked the following items as "wasted." Each item includes its name, up to the last 3 usage statuses, and its average expiry time in days. Based on this data:
Item data:
{item_data}
- Identify waste patterns.
- Suggest reduced purchase frequency if the item is frequently wasted.
- Recommend skipping items that are consistently wasted.
- If average expiry is low and item is often wasted, highlight perishability issues.
- If there's a mix of used and wasted statuses, suggest buying only when needed.
- Return clear personalized suggestions.

Strictly respond with a JSON object where:
- Each key is the item name.
- Each value is a suggestion string.

Do not include any explanation or formatting, just the raw JSON.
"""
)


In [42]:
chain_wasted_suggestions = prompt_template_waste | gemini | StrOutputParser()

In [43]:
result = chain_wasted_suggestions.invoke(input={ "item_data" : """[
  {
    "item_name": "Baby Spinach",
    "current_status": "wasted",
    "past_statuses": ["wasted", "used", "wasted"],
    "average_expiry_days": 4
  },
  {
    "item_name": "Milk",
    "current_status": "wasted",
    "past_statuses": ["wasted", "wasted", "used"],
    "average_expiry_days": 5
  },
  {
    "item_name": "Zucchini",
    "current_status": "wasted",
    "past_statuses": ["used", "wasted"],
    "average_expiry_days": 3
  },
  {
    "item_name": "Halloumi Cheese",
    "current_status": "wasted",
    "past_statuses": ["used", "used", "used"],
    "average_expiry_days": 20
  }
]""" })

In [44]:
# Clean markdown formatting
cleaned = re.sub(r"^```json|```$", "", result.strip(), flags=re.IGNORECASE).strip("`\n ")

waste_suggestion = json.loads(cleaned)
print(json.dumps(waste_suggestion, indent=2))

{
  "Baby Spinach": "Reduce purchase frequency of Baby Spinach as it's often wasted. Its short shelf life of 4 days contributes to spoilage. Consider buying smaller quantities more often.",
  "Milk": "Reduce purchase frequency of Milk as it's often wasted. Its short shelf life of 5 days contributes to spoilage. Consider buying smaller quantities more often.",
  "Zucchini": "Buy Zucchini only when needed.  Its short shelf life of 3 days contributes to spoilage.",
  "Halloumi Cheese": "Consider skipping Halloumi Cheese. Despite its longer shelf life, it has been consistently wasted recently."
}


In [72]:
prompt_template_recipe = PromptTemplate(
    input_variables=["meal", "item_list", "days_to_expiry"],
    template="""
    1. You are an AI nutritionist. Generate a recipe based on the following details:
    - Meal: {meal}
    - Items: {item_list}
    - Time until expiry: {days_to_expiry}
    2. The recipe should be optimized to minimize food waste by prioritizing items that expire soon. If absolutely needed, include items not available in the items list as well, while specifying them.
    3. Double check your responses before finalizing the meal recipe.
    4. Output the plan in JSON format with "recipes" array containing recipe_name, ingredients (item, quantity, note) and instructions, all for each recipe.
    5. Make sure that the response does not contain any preamble. Return JSON text only.
    """ )

In [73]:
chain_recipe = prompt_template_recipe | gemini | StrOutputParser()

In [74]:
result = chain_recipe.invoke(input={
    "meal": "Spinach and Tomato Omelette (using chickpea flour) with a side of lentils", "item_list":["rice", "lentils", "spinach", "tomatoes"],
        "days_to_expiry":calculate_days_to_expiry({"rice": "2025-04-10", "lentils": "2025-04-30", "spinach": "2025-04-18", "tomatoes": "2025-04-18"})
})

In [76]:
# Clean markdown formatting
cleaned = re.sub(r"^```json|```$", "", result.strip(), flags=re.IGNORECASE).strip("`\n ")

recipe = json.loads(cleaned)
print(json.dumps(recipe, indent=2))

{
  "recipes": [
    {
      "recipe_name": "Spinach and Tomato Chickpea Flour Omelette",
      "ingredients": [
        {
          "item": "Spinach",
          "quantity": "1 cup, chopped",
          "note": "Use fresh spinach as it expires sooner."
        },
        {
          "item": "Tomatoes",
          "quantity": "1/2 cup, diced",
          "note": "Use fresh tomatoes as they expire sooner."
        },
        {
          "item": "Chickpea flour",
          "quantity": "1/2 cup",
          "note": "Not in the provided items list, but essential for the omelette."
        },
        {
          "item": "Water",
          "quantity": "1/4 cup",
          "note": "Not in the provided items list."
        },
        {
          "item": "Onion",
          "quantity": "1/4 cup, chopped",
          "note": "Not in the provided items list, but recommended for flavor.  Optional."
        },
        {
          "item": "Garlic",
          "quantity": "1 clove, minced",
          "note":